<a href="https://colab.research.google.com/github/ahtyamovdanil/DLS_NLP/blob/master/%5Bhomework%5Dseq2seq_and_attn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hausaufgabe
translate(Hausaufgabe) -> **Homework**

Halo!
На семинаре мы создали простую seq2seq модель на основе rnn для перевода, а сейчас постараемся засунуть туда attention. Работать будем с тем же датасетом DE->EN (датасеты получше просто не влезают в память колаба, но если у вас есть CPU+тонна времени или GPU побольше, то можно попробовать построить перевод на WMT14 или IWSLT )

В конце домашней работы предполагается написание отчета о проделанной работе.

In [0]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import spacy

import random
import math
import time

from torchtext.datasets import TranslationDataset, Multi30k #WMT14, IWSLT
from torchtext.data import Field, BucketIterator

import torch.nn.functional as F

In [0]:
seed = 43

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
! python -m spacy download en
! python -m spacy download de


spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
     |████████████████████████████████| 14.9MB 1.7MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907056 sha256=f9378ea98d902414c261842831dc413a803565b38bc67a5f449230a1752548b5
  Stored in directory: /tmp/pip-ephem-wheel-cache-77s_gv8l/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [0]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

# немецкий язык является полем SRC, а английский в поле TRG
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [5]:
# В датасете содержится ~ 30к предложений средняя длина которых 11
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),  fields = (SRC, TRG))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:03<00:00, 318kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 92.4kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 88.2kB/s]


Давайте посмотрим что у нас с датасетом и сделаем словари для SRC и TGT

In [6]:
labels = ['train', 'validation', 'test']
dataloaders = [train_data, valid_data, test_data]
for d, l in zip(dataloaders, labels):
    print("Number of sentences in {} : {}".format(l, len(d.examples)))


Number of sentences in train : 29000
Number of sentences in validation : 1014
Number of sentences in test : 1000


In [7]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print("Number of words in source vocabulary", len(SRC.vocab))
print("Number of words in source vocabulary", len(TRG.vocab))

Number of words in source vocabulary 7855
Number of words in source vocabulary 5893


## Encoder

Энкодер будет ровно как в семинаре, с кдинственным изменением -- forward будет возвращать не только hidden, cell, но еще и outputs. Это нужно (надеюсь, вы уже поняли) для использования attention в декодере

In [0]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        """
        :param: input_dim is the size/dimensionality of the one-hot vectors that will be input to the encoder. This is equal to the input (source) vocabulary size.
        :param: emb_dim is the dimensionality of the embedding layer. This layer converts the one-hot vectors into dense vectors with emb_dim dimensions.
        :param: hid_dim is the dimensionality of the hidden and cell states.
        :param: n_layers is the number of layers in the RNN.
        :param: percentage of the dropout to use
        
        """
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding =  nn.Embedding(input_dim, emb_dim)
 
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        """
        :param: src sentences (src_len x batch_size)
        """
        # embedded = <TODO> (src_len x batch_size x embd_dim)
        embedded = self.embedding(src)
        # dropout over embedding
        embedded = self.dropout(embedded)
        outputs, (hidden, cell) = self.rnn(embedded)
        # [Attention return is for lstm, but you can also use gru]
        return outputs, hidden, cell

## Decoder
Оп ля, а тут уже что-то новенькое

Мы будем реализовывать attention, который будет смотреть из tgt в src (НЕ self-attention). 

Определим два класса -- Attention и DecoderAttn. Мы разбили их на два класса, чтобы можно было играться с типом внимания, не меняя код DecoderAttn. Как вы помните с лекции, в качестве аттеншена можно брать любую странную функцию (конкатенация, маленькая сеточка, ...), и все будет работать! Поэтому вам предлагается попробовать несколько разных.


---------------------
Есть два подхода к реализации аттеншена:

Подход #1:

1. Вычисляется embed
2. На основе hidden c прошлого шага, embedded и (возможно) enc_out вычисляется attention, а точнее, веса attention (поэтому не забудьте softmax!!). Размерность batch_size * max_len, max_len -- максимальная длина предложения в батче, т.е. shape[0] от выхода энкодера.
3. К enc_out применяется attention: чаще всего dot product от enc_out и attention_weights (не забудьте про измерение батч. Чтобы нормально вычислить dot_product по батчу, вам поможет torch.bmm)
4. Берутся attention и embedded и сворачиваются в один вектор размерности такой, чтобы кормить его self.lstm. Например, это можно сделать с помощью обычного линейного слоя
5. Вычисляется новое скрытое состояние new_hidden. Это наша self.lstm, примененная к выходу пункта 4.
6. Вычисляется prediction, как в семинаре

Грубо говоря, вся разница с семинаром в том, что мы вместо того, чтобы embedded пихать в self.lstm, миксуем аттэншен на основе всего, что имеем (enc_out, hidden, embedded) и запихиваем в self.lstm микс аттэншена и embedded.

![alt text](https://i.imgur.com/cmkRY0r.png)


Подход #2:

1. Вычисляется embed
2. Вычисляется output, new_hidden (строчка output, (hidden, cell) = self.rnn(embedded, (hidden, cell)))
3. На основе output и enc_out вычисляется attention, а точнее, веса attention (поэтому не забудьте softmax!!)
3. К enc_out применяется attention: чаще всего dot product от enc_out и attention_weights (не забудьте про измерение батч. Чтобы нормально вычислить dot_product по батчу, вам поможет torch.bmm)
4. Вычисляется prediction на основе attention и output. Можно, например, взять nn.Linear() от конкатенации attention и output.

Разница с первым подходом в том, что мы сначала вычисляем выход rnn слоя, а потом смотрим вниманием на src и на основе выхода rnn и attn считаем выход (prediction). 

![alt text](https://i.imgur.com/5aWjQWv.png)


Вам предлагается реализовать хотя бы 1 из вариантов и хотя бы 2 варианта функции attention (в классе Attention)


In [0]:
class Attention(nn.Module):
    def __init__(self, hidden_dim, method='additive'): # add parameters needed for your type of attention
        super(Attention, self).__init__()
        self.method = method # attention method you'll use. e.g. "cat", "one-layer-net", "dot", ...
        
        self.attn = nn.Linear(2*hidden_dim, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias = False)
        
    def forward(self, last_hidden, encoder_outputs, seq_len=None):
        
         
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        if self.method == 'additive':
            hidden = last_hidden.repeat(src_len, 1, 1)
            hidden = hidden.permute(1, 0, 2)
        elif self.method == 'dot':
            #hidden = last_hidden.unsqueeze(1)
            hidden = last_hidden.permute(1, 0, 2)
            #print(hidden.shape)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim]
        
        if self.method == 'additive':
            energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
            attention = self.v(energy).squeeze(2)
        elif self.method == 'dot':
            energy = torch.bmm(encoder_outputs, hidden.transpose(1,2)) 
            attention = energy.squeeze(-1)
        
        #energy = [batch size, src len, dec hid dim]      
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [57]:
hidden = torch.tensor([[10, 5, 10]]).float()
encoder_hidden = torch.tensor([[[0, 1, 1],[5, 0, 1],[1, 1, 0],[0, 5, 1]]]).permute(1,0,2).float()
att = Attention(3, method='dot')
print(encoder_hidden.shape)
att(hidden, encoder_hidden)

torch.Size([4, 1, 3])


RuntimeError: ignored

In [0]:
class DecoderAttn(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, attention, dropout=0.1):
        super(DecoderAttn, self).__init__()
        
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        
        self.attn = attention # instance of Attention class

        # define layers
        self.embedding = nn.Embedding(self.output_dim, self.emb_dim)
        
        self.rnn = nn.LSTM(self.emb_dim, self.hid_dim, self.n_layers)  #(lstm embd, hid, layers, dropout)
        self.out = nn.Linear(2*self.hid_dim+emb_dim, self.output_dim)  # Projection :hid_dim x output_dim
        self.dropout = nn.Dropout(dropout)

        # more layers you'll need for attention
        #<YOUR CODE HERE>
        
    def forward(self, input, hidden, cell, encoder_outputs):
        # make decoder with attention
        # use code from seminar notebook as base and add attention to it

        # (1x batch_size)
        input = input.unsqueeze(0)
        
        # (1 x batch_size x emb_dim)
        embedded = self.embedding(input)# embd over input and dropout 
        embedded = self.dropout(embedded)
                
                
        a = self.attn(hidden, encoder_outputs)
        #a = [batch size, rc len]

        a = a.unsqueeze(1)
        #a = [batch size, 1, src len]

        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        #encoder_outputs = [batch size, src len, enc hid dim]

        weighted = torch.bmm(a, encoder_outputs)
        #weighted = [batch size, 1, enc hid dim]

        weighted = weighted.permute(1, 0, 2)
        #weighted = [1, batch size, enc hid dim]

        rnn_input = torch.cat((embedded, weighted), dim = 2)
        #rnn_input = [1, batch size, enc hid dim + emb dim]

        #print('++++++ hidden ++++++', hidden.shape)
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #sent len and n directions will always be 1 in the decoder
        
        # (batch_size x output_dim)
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)

        prediction = torch.cat((output, weighted, embedded), dim = 1)
        prediction = self.out(prediction) #project out of the rnn on the output dim 
        
        return prediction, hidden, cell

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Seq2Seq module

Здесь опять ничего не поменяется кроме того, что энкодер теперь возвращает свой output, а декодер его принимает

In [0]:
# we need that to put it first to the decoder in 'translate' method
BOS_IDX = SRC.vocab.stoi['<sos>']

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        # Hidden dimensions of encoder and decoder must be equal
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self._init_weights() 
        self.max_len=30
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        """
        :param: src (src_len x batch_size)
        :param: tgt
        :param: teacher_forcing_ration : if 0.5 then every second token is the ground truth input
        """
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        encoder_outputs, hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0, :]
        
        for t in range(1, max_len):
            
            output, hidden, cell = self.decoder(input, hidden, cell, encoder_outputs) #pass state and input throw decoder 
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)
        
        return outputs
    
    def translate(self, src):
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = []
        src = src.unsqueeze(1)
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        src = src.to(self.device)
        encoder_outputs, hidden, cell = self.encoder(src) # TODO pass src throw encoder
        
        #first input to the decoder is the <sos> tokens
        input = src[0, :] # trg[idxs]
        
        for t in range(1, self.max_len):
            
            output, hidden, cell = self.decoder(input, hidden, cell, encoder_outputs) #TODO pass state and input throw decoder 
            top1 = output.max(1)[1]
            outputs.append(top1)
            input = (top1)
        
        return outputs
    
    def _init_weights(self):
        p = 0.08
        for name, param in self.named_parameters():
            nn.init.uniform_(param.data, -p, p)
        


In [0]:
from tqdm.notebook import tqdm

In [0]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(tqdm(iterator)):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing !!
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)


            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
src_embd_dim =  tgt_embd_dim = 256
hidden_dim = 512
num_layers =  1
dropout_prob = 0.2

batch_size = 64
PAD_IDX = TRG.vocab.stoi['<pad>']

iterators = BucketIterator.splits((train_data, valid_data, test_data),
                                  batch_size = batch_size, device = device)
train_iterator, valid_iterator, test_iterator = iterators

enc = Encoder(input_dim, src_embd_dim, hidden_dim, num_layers, dropout_prob)
attention = Attention(hidden_dim, method='dot')
dec = DecoderAttn(output_dim, tgt_embd_dim, hidden_dim, num_layers, attention, dropout_prob)
model = Seq2Seq(enc, dec, device).to(device)

In [64]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): DecoderAttn(
    (attn): Attention(
      (attn): Linear(in_features=1024, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512)
    (out): Linear(in_features=1280, out_features=5893, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

In [65]:
#### train model with additive method ####

max_epochs = 30
CLIP = 1

# TODO
optimizer = optim.Adam(model.parameters(), lr = 1e-4)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

best_valid_loss = float('inf')

for epoch in range(max_epochs):
    
    
    train_loss = round(train(model, train_iterator, optimizer, criterion, CLIP), 5)
    valid_loss = round(evaluate(model, valid_iterator, criterion), 5)
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    print('Train Perplexity {}  Val Perplexity {}:'.format(np.exp(train_loss), np.exp(valid_loss)))



Epoch: 0 
 Train Loss 5.24782  Val loss 4.98442:
Train Perplexity 190.15128648808474  Val Perplexity 146.11880153932967:



Epoch: 1 
 Train Loss 4.67595  Val loss 4.93747:
Train Perplexity 107.33448643480459  Val Perplexity 139.41707777930353:



Epoch: 2 
 Train Loss 4.44573  Val loss 4.84557:
Train Perplexity 85.26209645517632  Val Perplexity 127.17575151632522:



Epoch: 3 
 Train Loss 4.26656  Val loss 4.70336:
Train Perplexity 71.27602388707427  Val Perplexity 110.31721627705134:



Epoch: 4 
 Train Loss 4.09197  Val loss 4.62613:
Train Perplexity 59.85769528923502  Val Perplexity 102.11810136449044:


...


Epoch: 25 
 Train Loss 2.54999  Val loss 3.94966:
Train Perplexity 12.806975712265562  Val Perplexity 51.91771181163162:



Epoch: 26 
 Train Loss 2.48682  Val loss 3.89361:
Train Perplexity 12.022982182014102  Val Perplexity 49.08777391372034:



Epoch: 27 
 Train Loss 2.45619  Val loss 3.88386:
Train Perplexity 11.660301054331896  Val Perplexity 48.61149375685219:



Epoch: 28 
 Train Loss 2.40386  Val loss 3.90531:
Train Perplexity 11.065808067693977  Val Perplexity 49.6654738228828:



Epoch: 29 
 Train Loss 2.37204  Val loss 3.92588:
Train Perplexity 10.71923723864844  Val Perplexity 50.6976723861566:


In [67]:
test_loss = evaluate(model, test_iterator, criterion)

print('| Test Loss: {} Test PPL:{}|'.format(test_loss, np.exp(test_loss)))

| Test Loss: 3.946355313062668 Test PPL:51.74642321107816|


In [0]:
EOS_IDX = SRC.vocab.stoi['<eos>']

def translate(model, sentence):
    """
    function that uses .translate() method of the model to translate german sentence into english
    params:
        sentence: tokenized gernam sentence
    """
    sentence = sentence.lower()
    sent_vec = torch.tensor([SRC.vocab.stoi['sos']] + [SRC.vocab.stoi[token] for token in sentence.split()] + [SRC.vocab.stoi['eos']])
    #sent_vec = torch.tensor([SRC.vocab.stoi[token] for token in sentence.split()])
    #print(sent_vec)
    translation_idx = model.translate(sent_vec)
    res = []
    for t in translation_idx:
        if t[0] != EOS_IDX:
            #print(TRG.vocab.itos[t[0]], end=' ')
            res.append(TRG.vocab.itos[t[0]])
            #print(t[0].item(), end=' ')
        else:
            break
    return ' '.join(res)

In [69]:
translate(model, "ein klein apfel")

'a blue coat is standing on the . .'

In [94]:
n = 83
print(' '.join(valid_data[n].trg))
translate(model, ' '.join(valid_data[n].src))
#translate(' '.join(reversed(valid_data[n].src)))

a person with a purple shirt is painting an image of a woman on a white wall .


'a person in a white shirt is a a picture of a woman in a white wall .'

ИИИИ давайте также научимся считать самую популярную метрику для перевода -- BLEU (https://en.wikipedia.org/wiki/BLEU)

В общем-то, вам повезло -- ее писать руками скучно, да и nltk ее написало за вас:

In [0]:
from nltk.translate.bleu_score import corpus_bleu
def compute_bleu(model, inp_lines, out_lines):
    """ Estimates corpora-level BLEU score of model's translations given inp and reference out """
    translations = [translate(model, line) for line in inp_lines]
    return corpus_bleu([[ref] for ref in out_lines], translations) * 100

In [0]:
bleu_src = [' '.join(b.src) for b in test_data]
bleu_trg = [' '.join(b.trg) for b in test_data]

In [97]:
compute_bleu(model, bleu_src, bleu_trg)

37.618458933797804

Если вы реализовали несколько методов аттеншена, опишите каждый из них и полученную метрику на нем в отчете.

<Место для вашего отчета>